# Langchain X PaLM

This notebook contains the code for the LangChain + PaLM blog articles available on https://yactouat.com/.

## Setting things up

In [47]:
from dotenv import load_dotenv
from google.cloud import aiplatform
import langchain
from langchain import PromptTemplate
from langchain.chat_models import ChatVertexAI
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import numpy as np
import os
import time
# supporting type hints
from typing import List
import vertexai

load_dotenv()

True

In [48]:
# `GCP_LOCATION` is for instance `europe-west1`
vertexai.init(project=os.getenv('GCP_PROJECT_ID'), location=os.getenv('GCP_LOCATION')) 

print(f"Vertex AI SDK version: {aiplatform.__version__}")

print(f"LangChain version: {langchain.__version__}")

Vertex AI SDK version: 1.37.0
LangChain version: 0.0.346


In [49]:
# defining a function to rate limit API calls
def rate_limit(max_per_minute):
    period = 60 / max_per_minute
    print("Waiting")
    while True:
        before = time.time()
        # the `yield` keyword here controls back to the caller;
        # e.g. it allows the caller to perform the action that is rate limited
        yield
        # at this points, control returns to this function
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elapsed)
        if sleep_time > 0:
            print(".", end="")
            time.sleep(sleep_time)

In [50]:
class CustomVertexAIEmbeddings():
    def __init__(self, requests_per_minute: int, num_instances_per_batch: int):
        self.requests_per_minute = requests_per_minute
        self.num_instances_per_batch = num_instances_per_batch

    # takes a list of strings and returns a list of embeddings
    def embed_documents(self, texts: List[str]):
        limiter = rate_limit(self.requests_per_minute)
        results = []
        remaining = list(texts)

        while remaining:
            # working in batches because the API accepts maximum 5
            # documents per request to get embeddings
            batch, remaining = (
                # batch contains the first `num_instances_per_batch` documents
                remaining[: self.num_instances_per_batch],
                # docs contains the remaining documents
                remaining[self.num_instances_per_batch :],
            )
            self.vertex_ai_embeddings = VertexAIEmbeddings()
            chunk = self.vertex_ai_embeddings.client.get_embeddings(batch)
            results.extend(chunk)
            next(limiter)

        return [r.values for r in results]

In [51]:
# create a foundational LLM instance
# LLM model
llm = VertexAI(
    model_name="text-bison-32k@002",
    max_output_tokens=8192,
    # setting the temperature that low will make the mode more deterministic
    temperature=0.5,
    # (nucleus sampling): the model will only consider a cumulative probability threshold above 0.8 to consider words to use
    top_p=0.8,
    # the model will only consider the top k most likely words when generating text
    top_k=40,
    verbose=True,
)


In [52]:
# set embeddings
# `QPM` stands for "queries per minute"
EMBEDDING_QPM = 100
EMBEDDING_NUM_BATCH = 5
embeddings = CustomVertexAIEmbeddings(
    requests_per_minute=EMBEDDING_QPM,
    num_instances_per_batch=EMBEDDING_NUM_BATCH,
)

## Basic interactions with an LLM

Text is the natural way of interacting with LLMs, so let's just do that:

In [53]:
interaction = "what is the future of AI multi modality?"
response = llm(interaction)

response

' The future of AI multimodality holds immense potential for revolutionizing various industries and transforming how we interact with technology. Here are some key trends and developments to watch out for:\n\n1. **Enhanced Human-Machine Interaction**: AI multimodality will continue to enhance human-machine interaction by enabling seamless communication and collaboration between humans and machines. This will involve advancements in natural language processing, speech recognition, gesture recognition, and other multimodal interfaces.\n\n2. **Immersive Experiences**: AI multimodality will play a crucial role in creating immersive and engaging experiences across various domains. This includes virtual reality (VR), augmented reality (AR), and mixed reality (MR), where users can interact with digital content in a more natural and intuitive way.\n\n3. **Cross-Modal Understanding**: AI models will become increasingly adept at understanding and interpreting information from different modalitie

## Chat interactions

In [54]:
chat = ChatVertexAI(model_name="chat-bison-32k@002")

initial_human_message = HumanMessage(
    content="What is the future of AI multi modality?",
)

chat_interaction = [
    SystemMessage(
        content="You are an AI assistant that is specialized in software enginering and in machine learning. Your answers are two short sentences long maximum.",
    ),
    initial_human_message
]

response = chat(chat_interaction)

print(response.content)

 Multimodal AI systems will become increasingly sophisticated, enabling them to process and understand a wider range of data types, including text, images, audio, and video. This will allow them to perform a broader range of tasks, such as generating creative content, providing personalized recommendations, and making complex decisions.


In [55]:
response_with_history = chat([
    SystemMessage(
        content="You are an AI assistant that is specialized in software enginering and in machine learning. Your answers are two short sentences long maximum.",
    ),
    initial_human_message,
    AIMessage(
        content=response.content,
    ),
    HumanMessage(content="what wider range of data types?")
])

print(response_with_history.content)

 Multimodal AI systems will be able to process and understand a wider range of data types, including text, images, audio, video, and even haptic data. This will allow them to interact with the world in a more natural and intuitive way.


## Embeddings basics

In [56]:
# here's how you can get an embedding for a sentence
text = "I love Shakespeare"
text2 = "Life without theaters would be dull"
# using a completely unrelated sentence
text3 = "Hey honey where did you put the keys?"

# sending the text to create embeddings for to Vertex AI
text_embeddings = embeddings.embed_documents([text, text2, text3])
# showing only the first 5 values of each embedding
text_embeddings_short = [e[:5] for e in text_embeddings]
text_embeddings_short

Waiting


[[-0.033491309732198715,
  -0.024705076590180397,
  0.04814573749899864,
  -0.0027620443142950535,
  -0.006553742568939924],
 [-0.011842082254588604,
  0.0037548220716416836,
  -0.010157657787203789,
  -0.009990965016186237,
  0.03311667591333389],
 [0.018089881166815758,
  -0.012067587114870548,
  0.02881603129208088,
  0.05261128023266792,
  -0.003405238501727581]]

In [57]:
# checking the cosine similarity between the first two embeddings
euclidian_distance_1_2 = np.linalg.norm(np.array(text_embeddings[0]) - np.array(text_embeddings[1]))
# now between the first and the third
euclidian_distance_1_3 = np.linalg.norm(np.array(text_embeddings[0]) - np.array(text_embeddings[2]))

euclidian_distance_1_2, euclidian_distance_1_3
# we can see that the first two embeddings are closer to each other than the first and the third
# ! watch out, this is just an example as the magnitude of the embeddings is not normalized

(0.9161093776749432, 1.0209345760221844)

In [58]:
# now let's do the same thing using cosine similarity
cosine_similarity_1_2 = np.dot(text_embeddings[0], text_embeddings[1]) / (
    np.linalg.norm(text_embeddings[0]) * np.linalg.norm(text_embeddings[1])
)
cosine_similarity_1_3 = np.dot(text_embeddings[0], text_embeddings[2]) / (
    np.linalg.norm(text_embeddings[0]) * np.linalg.norm(text_embeddings[2])
)

cosine_similarity_1_2, cosine_similarity_1_3

(0.5803716847710505, 0.4788461710635313)

## Prompt templates

In [59]:
template = """
What do you think of this technology {tech}? Is it still relevant for use today?

Your answers are two short sentences long maximum.
"""

prompt = PromptTemplate(
    input_variables=["tech"],
    template=template,
)

final_prompt = prompt.format(tech="steam engine")
response = llm(final_prompt)
print(response)


 The steam engine, a pivotal invention of the Industrial Revolution, revolutionized industries and transportation. While its significance cannot be overstated, its relevance in today's world is limited due to the advent of more efficient and environmentally friendly technologies.
